In [ ]:
%matplotlib inline
import matplotlib
import seaborn as sns
matplotlib.rcParams['savefig.dpi'] = 2 * matplotlib.rcParams['savefig.dpi']

# Time Series

Time series prediction presents its own challenges which are different from machine-learning problems.  Like many other classes of problems, it also presents a number of special features which are common.

To demonstrate this, we will use a weather dataset.  The basic setup for it is in `projects/weather`.  Use the makefile to download data.

## Cross-validation is different
Cross validation is very different for time series than with other machine-learning problem classes.  In normal machine-learning, we select a random subset of data as a validation set to estimate accuracy of your measure.  In time series, we have to consider the problem that we are trying to solve is often to predict a value in the future.  Therefore, the validation data always has to occur *after* the training data.  As a simple example, consider that it would not be very useful to have a predictor of tomorrow's temperature that depended on the temperature the day after. In general:

- We cannot just pick data points at random, because there might be lagged indicators / seasonal effects / etc. that force us to work with contiguous blocks of time.  
- We cannot blindly chop by e.g., month or year without some thought: There could be seasonal effects so that Decembers are always different.  There could be systemic "regime changes" that mean that cutting at a given date is inappropriate, or known and time-limited effects that last a year (or fraction thereof).   For instance, the years 1991 and 2008 in this data set.
- We cannot have our testing set occur before our training set.

![ts cross-validation illustration](images/ts_xval.png)

We usually handle this by doing a **sliding-window validation method**.  That is, we train on the last $n$ data points and validate the prediction on the next $m$ data points, sliding the $n + m$ training / validation window in time.  In this way, we can estimate the parameters of our model.  To test the validity of the model, we might use a block of data at the end of our time series which is reserved for testing the model with the learned parameters.

Another common technique is to use [**forward chaining**](http://stats.stackexchange.com/questions/14099/using-k-fold-cross-validation-for-time-series-model-selection).

*Question:* What are the strengths and weaknesses of using a fixed vs. rolling window?

Another concern is whether the time series results are predictive.  In economics, we refer to this as the ergodicity assumption, that past behavior can inform future behavior.  Many wonder if past behavior in daily stock returns gives much predictive power for future behavior.

**Warning**: Feature generation is sometimes a little different for time-series.  Usually, feature generation on a set is only based on data in that training example (e.g. extracting the time of day of the temperature measurement).  In timeseries, we often want to use *lagged* data (the temperature an hour ago).  The easiest way to do this is to do the feature generation *before* making the training and validation split.

**Exercise**: Split the data into a training and validation set.  Use the last year of data for training.  Using the 'mean temperature' (`dummy.DummyRegressor`) as a baseline model.

## Features special to time-series

### Seasonality:
There are two ways to handle seasonality.  Seasonality features are nice because they are good at projecting arbitrarily far into the future.

1. The simplest (and perhaps most robust) is to have a set of indicator variables for each month.  **Question**: should month be a continuous or categorical variable?
1. Since we know that temperature is roughly sinusoidal, we know that a reasonable model might be
$$ y_t = k \sin\left( \frac{t - t_0}{T} \right) + \epsilon$$
where $k$ and $t_0$ are parameters to be learned and $T$ is one year for seasonal variation.  While this is linear in $k$, it is not linear in $t_0$.  However, we know from Fourier analysis, that the above is equivalent to 
$$ y_t = A \sin\left( \frac{t}{T} \right) + B \cos\left( \frac{t}{T} \right) + \epsilon$$
which is linear in $A$ and $B$.  This can be solved using a linear regression.

**Exercise**: Graph temperature as a function of time on both the annual and daily levels.  Build transformers that generate these features and use `pipeline.Pipeline` to concatonate them with a regressor like `linear_model.LinearRegressor`.  How do they perform?

### Last data point:
Often in time series, the "last data point", is a good model for the next value.  Simple variations on this include [exponential moving averages](https://en.wikipedia.org/wiki/Moving_average#Exponential_moving_average) and [rolling window averages](https://en.wikipedia.org/wiki/Moving_average#Cumulative_moving_average).  While these features might give more accurate data, they are only useful for short-term projections wherreas seasonality is able to predict longer-term trends.

**Exercise**: For weather, there are two useful "last data points".  The first is the temperature in the last hour.  The second is the temperature 24 hours ago.  Build these as features and see how they improve performance.
*Note:*
1.  These are features that should probably be generated on the entire dataset rather than separately on the training and validation sets (can you explain why?)
1.  Be careful in computing these to make sure you don't accidently have information about today's temperature in your correlates (this would be cheating!).

### Auto-correlation:
Plot the [auto-correlation](https://en.wikipedia.org/wiki/Autocorrelation) of the residual (the unexplained portion of the data after taking the best fit with one of your above models).  There are [pandas functions](http://pandas.pydata.org/pandas-docs/stable/visualization.html#autocorrelation-plot) that make this easier.  To handle remaining autocorrelation, there is an entire literature on [auto-regressive models](https://en.wikipedia.org/wiki/Autoregressive_model) as well as [auto-regressive moving average models](https://en.wikipedia.org/wiki/Autoregressive%E2%80%93moving-average_model).  Read up about them on wikipedia to get a better idea.  In python, they are implemented in [Statsmodels](https://github.com/statsmodels/statsmodels/).

**Exercise**: Write a *residual regressor* that takes your best model and tries to fit and AR or ARMA model on the residuals using Statsmodels.  *Notes:*
1.  We should always first remove seasonality and "last data point" features (the low-hanging fruit) before trying to compute auto-correlation on the residuals.
1.  It is not always natural to fit statsmodels into the framework
1.  While the previous models could be tested by training models on the entire training set and then validating, for these autocorrelation models, we have to use sliding-window validation method mentioned above.

### Other correlates:
So far, our features have all depended on the date and previous temperature.  However, we can add in other variables into our prediction.

**Exercise**: Build a temperature predictor using just these other features.  Try experimenting with non-linear models like random forests as well as linear ones.  While it may not be as effective, it can add signal when combined with the above models.  *Notes*:
1. Make sure you don't accidently give yourself access to concurrent data: you cannot use the wind direction this hour to predict the temperature, you have to use past wind direciton information.
1. Try combining the signal using non-linear as well as linear techniques.  Is there a reason you might think that linear techniques work pretty well for aggregation of different signals?

*Copyright &copy; 2015 The Data Incubator.  All rights reserved.*